# ML Assignment 2: Optimization

## Overview

In developing this script, I referenced
 Hayes, G. (2019). mlrose: Machine Learning, Randomized Optimization and SEarch package for Python. https://github.com/gkhayes/mlrose 
 and also https://github.com/hiive/mlrose

mlrose is a Python package for applying some of the most common randomized optimization and search algorithms to a range of different optimization problems, over both discrete- and continuous-valued parameter spaces. This notebook contains the examples used in the mlrose tutorial.

### Import Libraries

In [1]:
import time
import threading

import mlrose_hiive as mlrose
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

### Problem 1:  Fill the Knapsack!

In [2]:
class Runner:
    def __init__(self,n, problem_name, algs, alg_names):
        self.n = n
        self.name = problem_name
        fig1, ax1 = plt.subplots()
#         ax1.title = "Score Per Iteration"
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
#         ax2.title = "Evals Per Iteration"
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.algs = algs
        self.names = alg_names
        self.times = []
        self.best_fitnesses = []
        
    def run(self):
        for algorithm, name in zip(self.algs,self.names):
            print(f'starting {name}....')
            tic = time.perf_counter()
            best_state, best_fitness, fitness_curve = algorithm()
            toc = time.perf_counter()
#             print(f"{name} completed in {toc - tic:0.4f} seconds")
#             print(f'{name} found best fitness/value: ', best_fitness, '\n')
            
            self.times.append(toc-tic)
            self.best_fitnesses.append(best_fitness)
            self.plot_curve(fitness_curve, f'{name}')
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    
    def print_stats(self):
        for name, time, best_fitness in zip(self.names, self.times, self.best_fitnesses):
            print(f'{name} had \n\tTime: {time:0.4f} seconds\n\tFitness Score: {best_fitness}')
    def plot_curve(self, fitness_curve, name):
        self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
        self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [4]:
# Initialize fitness function object using pre-defined class
'''
Fitness function for Knapsack optimization problem. Given a set of n
items, where item i has known weight :math:`w_{i}` and known value
:math:`v_{i}`; and maximum knapsack capacity, :math:`W`, the Knapsack
fitness function evaluates the fitness of a state vector
:math:`x = [x_{0}, x_{1}, \ldots, x_{n-1}]` as:
'''
# https://en.wikipedia.org/wiki/Knapsack_problem
# We're trying to get to the highest value possible, without going over our weight limit

ns = [50,100,200,1000] # items in our knapsack
weights = []
values = []
fitnesses = []
init_states = []
for n in ns:
    weight = (np.random.randint(1,50,size=(n)))
    value = (np.random.randint(1,50,size=(n)))
    init_state = np.random.randint(0,1,size=(n))
    init_states.append(init_state)
    max_weight_pct = 0.6 # so we can only hold 60% of our total weight for items we're trying to fit
    fitnesses.append(mlrose.Knapsack(weight, value, max_weight_pct))
    weights.append(weight)
    values.append(value)

# basically have a seed state so we can reproduce
random_state = 5
for weight, value, fitness, n, init_state in zip(weights, values, fitnesses, ns, init_states):
    print('=====================')
    print(f'starting n = {n}...')
    algs = []
    alg_names = []
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    alg_names.append('random_hill_climbing')
    
    

    schedule = mlrose.GeomDecay(.01,0.0001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.simulated_annealing(problem, schedule=schedule, curve=True, random_state = random_state))
    alg_names.append('simulated_annealing')
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    
    alg_names.append('genetic_algorithm')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    alg_names.append('mimic')
    
    thing = Runner(n, 'knapsack', algs, alg_names)
    thing.run()
    thing.print_stats()
    print('\n')

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0015 seconds
	Fitness Score: 833.0
simulated_annealing had 
	Time: 0.0016 seconds
	Fitness Score: 842.0
genetic_algorithm had 
	Time: 0.5562 seconds
	Fitness Score: 1140.0
mimic had 
	Time: 0.4389 seconds
	Fitness Score: 1119.0


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0014 seconds
	Fitness Score: 1568.0
simulated_annealing had 
	Time: 0.0015 seconds
	Fitness Score: 1546.0
genetic_algorithm had 
	Time: 1.0966 seconds
	Fitness Score: 2081.0
mimic had 
	Time: 0.9030 seconds
	Fitness Score: 2035.0


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0034 seconds
	Fitness S

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Part 2: Six Peaks 

In [6]:
# Initialize fitness function object using pre-defined class

ns = [50,100,200,500] # items in our knapsack

fitnesses = []
for n in ns:
    fitnesses.append(mlrose.SixPeaks(t_pct=0.20))


# basically have a seed state so we can reproduce


# basically have a seed state so we can reproduce
random_state = 5
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs = []
    alg_names = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    alg_names.append('random_hill_climbing')
    
    

    schedule = mlrose.GeomDecay(.01,0.001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.simulated_annealing(problem,schedule=schedule, curve=True, random_state = random_state))
    alg_names.append('simulated_annealing')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    
    alg_names.append('genetic_algorithm')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    alg_names.append('mimic')
    
    thing = Runner(n, 'six_peaks', algs, alg_names)
    thing.run()
    thing.print_stats()
    print('\n')

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0003 seconds
	Fitness Score: 6.0
simulated_annealing had 
	Time: 0.0074 seconds
	Fitness Score: 76.0
genetic_algorithm had 
	Time: 0.7436 seconds
	Fitness Score: 83.0
mimic had 
	Time: 0.2450 seconds
	Fitness Score: 16.0


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0003 seconds
	Fitness Score: 1.0
simulated_annealing had 
	Time: 0.0989 seconds
	Fitness Score: 63.0
genetic_algorithm had 
	Time: 0.9114 seconds
	Fitness Score: 136.0
mimic had 
	Time: 0.7808 seconds
	Fitness Score: 22.0


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0003 seconds
	Fitness Score: 4.0
simul

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Example 3: Queens

In [7]:
ns = [50,100,200,500] # items in our knapsack

fitnesses = []
init_coords = []
for n in ns:
    fitnesses.append(mlrose.Queens())


# basically have a seed state so we can reproduce
random_state = 5
times = []
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs = []
    alg_names = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    alg_names.append('random_hill_climbing')
    
    schedule = mlrose.GeomDecay(.01,0.0001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.simulated_annealing(problem, curve=True, random_state = random_state))
    alg_names.append('simulated_annealing')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, max_iters=100, random_state=random_state))
    
    alg_names.append('genetic_algorithm')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    alg_names.append('mimic')
    
    thing = Runner(n, 'queens', algs, alg_names)
    thing.run()
    thing.print_stats()
    print('\n')


starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0437 seconds
	Fitness Score: 86.0
simulated_annealing had 
	Time: 0.2460 seconds
	Fitness Score: 90.0
genetic_algorithm had 
	Time: 3.0760 seconds
	Fitness Score: 88.0
mimic had 
	Time: 3.5402 seconds
	Fitness Score: 93.0


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0486 seconds
	Fitness Score: 154.0
simulated_annealing had 
	Time: 0.6331 seconds
	Fitness Score: 181.0
genetic_algorithm had 
	Time: 13.8467 seconds
	Fitness Score: 178.0
mimic had 
	Time: 9.1183 seconds
	Fitness Score: 184.0


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0740 seconds
	Fitness Score: 307

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Example 6: Fitting a Neural Network to the Heart Dataset

In [8]:

df = pd.read_csv("data/heart.csv")
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
 # https://www.kaggle.com/ronitf/heart-disease-uci


In [9]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 3)

In [10]:
# Normalize feature data
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# One hot encode target values
one_hot = OneHotEncoder()

y_train_hot = one_hot.fit_transform(np.array(y_train).reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(np.array(y_test).reshape(-1, 1)).todense()

In [28]:
class NNRunner:
    def __init__(self, alg_names):
        fig1, ax1 = plt.subplots()
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.names = alg_names
        self.name = "NN"
        self.times = []
        self.scores = []
        
    def run(self):
        for name in self.names:
            print(f'starting {name}....')
            tic = time.perf_counter()
            print('===============================================')
            print(f'running NN weight optimization for {name}')
            # Initialize neural network object and fit object - attempt 1


            schedule = mlrose.GeomDecay(.01,0.0001) 
            nn = mlrose.NeuralNetwork(hidden_nodes = [2], activation ='relu', 
                                             algorithm =name, 
                                             max_iters = 100, bias = False, is_classifier = True, 
                                             learning_rate = 0.0001, early_stopping = True,
                                             schedule=schedule,
                                             curve=True,
                                             clip_max = 5, max_attempts = 1000, random_state = 5)
            toc = time.perf_counter()
            
            nn.fit(X_train_scaled, y_train_hot)
    
            fitness_curve = nn.fitness_curve

            # Predict labels for train set and assess accuracy
            y_train_pred = nn.predict(X_train_scaled)

            y_train_accuracy = accuracy_score(y_train_hot, y_train_pred)

            print(y_train_accuracy)

            # Predict labels for test set and assess accuracy
            y_test_pred = nn.predict(X_test_scaled)

            y_test_accuracy = accuracy_score(y_test_hot, y_test_pred)

            print(y_test_accuracy)
            print('\n')
            
            self.times.append(toc-tic)
            self.scores.append(y_test_accuracy)
            self.plot_curve(fitness_curve, f'{name}')
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    
    def print_stats(self):
        for name, time, best_fitness in zip(self.names, self.times, self.best_fitnesses):
            print(f'{name} had \n\tTime: {time:0.4f} seconds\n\tFitness Score: {best_fitness}')
    def plot_curve(self, fitness_curve, name):
        if len(fitness_curve.shape) == 2:
            self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
            self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [29]:
algorithms = ['random_hill_climb', 'simulated_annealing', 'genetic_alg','gradient_descent']

runner_for_nn = NNRunner(algorithms)

runner_for_nn.run()

runner_for_nn.print_stats
    

    

starting random_hill_climb....
running NN weight optimization for random_hill_climb
0.6239669421487604
0.5573770491803278


starting simulated_annealing....
running NN weight optimization for simulated_annealing
0.6239669421487604
0.5573770491803278


starting genetic_alg....
running NN weight optimization for genetic_alg
0.8471074380165289
0.8360655737704918


starting gradient_descent....
running NN weight optimization for gradient_descent
0.7231404958677686
0.7213114754098361




<bound method NNRunner.print_stats of <__main__.NNRunner object at 0x13f437520>>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>